In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
import numpy as np
import cv2
import os
import tqdm

In [5]:
PATH_COVID = './dataset/covid/'
PATH_NORMAL = './dataset/normal/'

In [6]:
covid_list = os.listdir(PATH_COVID)
normal_list = os.listdir(PATH_NORMAL)

In [7]:
def load_reshape_img(image):
    img = load_img(image, target_size=(224, 224))
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)
    return x


In [8]:
datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    brightness_range=[0.2,1.0],
                                    shear_range=0.2,
                                    zoom_range=[0.5,1.0],
                                    fill_mode='nearest')

for xr in tqdm.tqdm(covid_list):
    X = np.array([load_reshape_img(PATH_COVID + image) for image in covid_list])
    X = X.reshape(X.shape[0], 224, 224, 3)    
    i = 0
    for batch in datagen.flow(X, batch_size=70,
                              save_to_dir=PATH_COVID, 
                              save_prefix='covid', 
                              save_format='jpeg'):        
        i += 1
        if i > 10:
            break


  3%|▎         | 2/70 [01:04<36:43, 32.41s/it]


KeyboardInterrupt: 

In [ ]:
for xr in tqdm.tqdm(normal_list):
    X = np.array([load_reshape_img(PATH_NORMAL + image) for image in normal_list])
    X = X.reshape(X.shape[0], 224, 224, 3)    
    i = 0
    for batch in datagen.flow(X, batch_size=70,
                              save_to_dir=PATH_NORMAL, 
                              save_prefix='normal', 
                              save_format='jpeg'):        
        i += 1
        if i > 10:
            break

In [20]:
model_v3 = InceptionV3(weights='imagenet', include_top=False, classes=2, input_shape=(224, 224, 3))
new_layers = model_v3.output

new_layers = GlobalAveragePooling2D()(new_layers)

new_layers = Dense(128, activation='relu')(new_layers)
new_layers = Dropout(0.5)(new_layers)
new_layers = BatchNormalization()(new_layers)

new_layers = Dense(2, activation='softmax')(new_layers)
model_v3 = Model(inputs=model_v3.inputs, outputs=new_layers)

In [21]:
# Freezing the first 51 layers
for layer in model_v3.layers[:52]:
    layer.trainable = False

In [22]:
model_v3.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy']) #Stochastic gradient descent optimizer.


In [58]:
train_datagen = ImageDataGenerator()
#                                     rescale=1./255,
#                                     rotation_range=40,
#                                     width_shift_range=0.2,
#                                     height_shift_range=0.2,
#                                     brightness_range=[0.2,1.0],
#                                     shear_range=0.2,
#                                     zoom_range=[0.5,1.0],
#                                     fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './dataset/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

# validation_generator = test_datagen.flow_from_directory(
#         './dataset/valid/', # same directory as training data
#         target_size=(224, 224),
#         batch_size=1,
#         class_mode='binary'
#   ) # set as validation data



# model_v3.fit(
#         train_generator,
#         steps_per_epoch=100,
#         epochs=2)
model_v3.fit_generator(
    train_generator,
    steps_per_epoch = 60,
    validation_data = validation_generator, 
    validation_steps = 10,
    epochs = 5)

Found 28106 images belonging to 2 classes.
  ...
    to  
  ['...']


ValueError: `validation_steps` should not be specified if `validation_data` is None.

In [23]:
train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    brightness_range=[0.2,1.0],
                                    shear_range=0.2,
                                    zoom_range=[0.5,1.0],
                                    fill_mode='nearest',
                                    validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        './dataset/',
        target_size=(224, 224),
        batch_size=16,
        class_mode='binary',
        subset='training')
validation_generator = train_datagen.flow_from_directory(
        './dataset/', # same directory as training data
        target_size=(224, 224),
        batch_size=16,
        class_mode='binary',
        subset='validation') # set as validation data

model_v3.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 16,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 16,
    epochs = 20)

Found 1872 images belonging to 2 classes.
Found 467 images belonging to 2 classes.
Epoch 1/20
117/117 [==============================] - 55s 471ms/step - loss: 0.9342 - accuracy: 0.5272 - val_loss: 0.4808 - val_accuracy: 0.8254
Epoch 2/20
117/117 [==============================] - 54s 459ms/step - loss: 0.7547 - accuracy: 0.6052 - val_loss: 0.4218 - val_accuracy: 0.8728
Epoch 3/20
117/117 [==============================] - 54s 459ms/step - loss: 0.6156 - accuracy: 0.6880 - val_loss: 0.3483 - val_accuracy: 0.9181
Epoch 4/20
117/117 [==============================] - 53s 457ms/step - loss: 0.5303 - accuracy: 0.7420 - val_loss: 0.2575 - val_accuracy: 0.9440
Epoch 5/20
117/117 [==============================] - 54s 458ms/step - loss: 0.4567 - accuracy: 0.7842 - val_loss: 0.4157 - val_accuracy: 0.8470
Epoch 6/20
117/117 [==============================] - 54s 457ms/step - loss: 0.3822 - accuracy: 0.8504 - val_loss: 0.2286 - val_accuracy: 0.9418
Epoch 7/20
117/117 [===========================

In [24]:
def save_model(model):
    # serialize model to JSON
    with open(f"{model}.json", "w") as json_file:
        json_file.write(model_v3.to_json())

    # serialize weights to HDF5
    model_v3.save_weights(f"{model}.h5")
    print("Saved model to disk")
save_model('model')    

Saved model to disk
